In [15]:
import json
import os
from pathlib import Path

error_count = 0
data_count = 0
for json_file in sorted(Path("./").glob("intermediate_results*.json")):
    print(f"Processing {json_file}")
    with open(json_file, "r") as f:
        data = json.load(f)
        # print((data[0].keys()))
        for paper in data:
            data_count += 1
            if len(paper['errors']) != 0:
                print(paper['provider'])
                print(paper['errors'])
                print('=' * 100)
                error_count += 1

print(f"Total error count: {error_count}")
print(f"Total data count: {data_count}")

        # data.keys()
    # for paper_id, paper_data in data.items():
        # print(f"Paper ID: {paper_id}")
        # print(f"Paper Data: {paper_data}")        
        # print("-" * 100)


Processing intermediate_results_10.json
Processing intermediate_results_100.json
gpt-oss:20b
['No relevant chunks found for criterion 1', 'No relevant chunks found for criterion 2', 'No relevant chunks found for criterion 3', 'No relevant chunks found for criterion 4', 'No relevant chunks found for criterion 5', 'No relevant chunks found for criterion 6']
gpt-oss:20b
['No relevant chunks found for criterion 1', 'No relevant chunks found for criterion 2', 'No relevant chunks found for criterion 3', 'No relevant chunks found for criterion 4', 'No relevant chunks found for criterion 5', 'No relevant chunks found for criterion 6']
gpt-4o
["Criterion 3 failed: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-NBvF0GcMlqUT7U6nMgdT0AOb on tokens per min (TPM): Limit 30000, Used 29009, Requested 2286. Please try again in 2.59s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exc

In [2]:
from minio import Minio
from io import BytesIO
client = Minio(
    "localhost:5000",
    access_key="minioadmin",
    secret_key="minioadmin",
    secure=False
)

bucket_name = "criteria-classified-articles"

if not client.bucket_exists(bucket_name):
    print(f"Bucket {bucket_name} does not exist. Creating it...")
    client.make_bucket(bucket_name)
else:
    print(f"Bucket {bucket_name} already exists.")


Bucket criteria-classified-articles already exists.


In [3]:
bucket_name

'criteria-classified-articles'

In [ ]:
files = [obj.object_name for obj in client.list_objects(bucket_name, recursive=False)]
paper_ids = [f.split("/", 1)[-1].rsplit(".json", 1)[0] if "/" in f else f.rsplit(".json", 1)[0] for f in files]
print(paper_ids)
with open("labeled_paper_ids.txt", "w") as f:
    for paper_id in paper_ids:
        f.write(f"{paper_id}\n")


['10081988', '10094530', '10160943', '10169927', '10231228', '10239947', '10354256', '10370087', '10435436', '10512116', '10571424', '10619638', '10636380', '10641704', '10652502', '10664788', '10712251', '10716572', '10720646', '10753978', '10754247', '10765675', '10771641', '10775534', '10779304', '10782027', '10830735', '10861529', '10870524', '10876777', '10889650', '10910118', '10938738', '10942681', '10948549', '10977422', '11004446', '11016626', '11034607', '11049980', '11056767', '11061024', '11074471', '11084258', '11087100', '11096934', '11109891', '11126816', '11130559', '11175770', '11175821', '11177169', '11180144', '11181307', '11183922', '11185664', '11196421', '11213077', '11214072', '11220500', '11227945', '11245480', '11278143', '11287771', '11289729', '11290140', '11299072', '11303238', '11314332', '11326595', '11358401', '11371670', '11386077', '11391518', '11401950', '11408257', '11409932', '11411091', '11416139', '11419182', '11430092', '11432175', '11449975', '70

In [24]:
for json_file in sorted(Path("./").glob("intermediate_results*.json")):
    print(f"Processing {json_file}")
    with open(json_file, "r") as f:
        data = json.load(f)
        for paper in data:
            if len(paper['errors']) == 0:
                json_data = json.dumps(paper).encode('utf-8')
                content_length = len(json_data)
                client.put_object(
                    bucket_name = bucket_name,
                    object_name = f"{paper['provider']}/{paper['paper_id']}.json",
                    data = BytesIO(json_data),
                    length = content_length,
                    content_type = "application/json"
                )

print("All data uploaded successfully.")

Processing intermediate_results_10.json
Processing intermediate_results_100.json
Processing intermediate_results_110.json
Processing intermediate_results_120.json
Processing intermediate_results_130.json
Processing intermediate_results_140.json
Processing intermediate_results_150.json
Processing intermediate_results_20.json
Processing intermediate_results_30.json
Processing intermediate_results_40.json
Processing intermediate_results_50.json
Processing intermediate_results_60.json
Processing intermediate_results_70.json
Processing intermediate_results_80.json
Processing intermediate_results_90.json
All data uploaded successfully.


In [25]:
for json_file in sorted(Path("./").glob("final_rag_labelling_results.json")):
    print(f"Processing {json_file}")
    with open(json_file, "r") as f:
        data = json.load(f)
        for paper in data:
            if len(paper['errors']) == 0:
                json_data = json.dumps(paper).encode('utf-8')
                content_length = len(json_data)
                client.put_object(
                    bucket_name = bucket_name,
                    object_name = f"{paper['provider']}/{paper['paper_id']}.json",
                    data = BytesIO(json_data),
                    length = content_length,
                    content_type = "application/json"
                )

print("All data uploaded successfully.")

All data uploaded successfully.


In [29]:
bucket_name = "raw-pubmed-articles"
if not client.bucket_exists(bucket_name):
    print(f"Bucket {bucket_name} does not exist. Creating it...")
    client.make_bucket(bucket_name)
else:
    print(f"Bucket {bucket_name} already exists.")

Bucket raw-pubmed-articles does not exist. Creating it...


In [30]:
with open('./processed_output.json', 'r') as f:
    data = json.load(f)
    # print(data.keys())
    for pid, paper in data.items():
        json_data = json.dumps(paper).encode('utf-8')
        content_length = len(json_data)
        client.put_object(
            bucket_name = bucket_name,
            object_name = f"{pid}.json",
            data = BytesIO(json_data),
            length = content_length,
            content_type = "application/json"
        )

print("All data uploaded successfully.")


All data uploaded successfully.
